##  The major task of this assignment to implement DenseNet for CIFAR 10 or CIFAR 100.

Here, I have implemented DenseNet for CIFAR 10 dataset.

In [ ]:
## importing libraries
import tensorflow as tf
import numpy as np

In [ ]:
## Load the CIFAR 10 dataset

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

## One-hot encoding for 10 classes.

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

170500096/170498071 [==============================] - 6s 0us/step


### The $Composite function$ ( $BatchNorm \to ReLU \to Conv$ )

composite_function represents a function which takes in an image/feature map ( $x$ ) and performs some operations on it.

$ x \to Batch\ Normalization \to ReLU \to Zero\ Padding \to 3 \times 3 \ Convolution\ \to Dropout$  
The bottleneck layer could be added too.

In [ ]:
def composite_function(  inputs, num_filters , dropout_rate ):
    x = tf.keras.layers.BatchNormalization( epsilon=eps )( inputs )
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.ZeroPadding2D((1, 1))(x)
    x = tf.keras.layers.Conv2D(num_filters, kernel_size=(3, 3), use_bias=False , kernel_initializer='he_normal' )(x)
    x = tf.keras.layers.Dropout(rate=dropout_rate )(x)
    return x

### The Transition Layers

The Transition layers perform the downsampling of the feature maps. The feature maps come from the previous block. The compression_factor observed below is the $\theta$ value from the paper which is the compression factor.

Hence, if $m$ feature maps go into the transition layer, we'll produce $[m \theta]$ feature maps.


In [ ]:

def transition(inputs, num_filters , compression_factor , dropout_rate ):
    # compression_factor is the 'θ'
    x = tf.keras.layers.BatchNormalization( epsilon=eps )(inputs)
    x = tf.keras.layers.Activation('relu')(x)
    num_feature_maps = inputs.shape[1] # The value of 'm'

    x = tf.keras.layers.Conv2D( np.floor( compression_factor * num_feature_maps ).astype( np.int ) ,
                               kernel_size=(1, 1), use_bias=False, padding='same' , kernel_initializer='he_normal' , kernel_regularizer=tf.keras.regularizers.l2( 1e-4 ) )(x)
    x = tf.keras.layers.Dropout(rate=dropout_rate)(x)

    x = tf.keras.layers.AveragePooling2D(pool_size=(2, 2))(x)
    return x

### The Dense Block

Each block will get some feature maps as input  from the previous transition layer. These inputs will then go through the $Composite Function$ to produce an output($x_1$ ) .

In [ ]:
def dense_block( inputs, num_layers, num_filters, growth_rate , dropout_rate ):
    for i in range(num_layers): # num_layers is the value of 'l'
        conv_outputs = composite_function(inputs, num_filters , dropout_rate )
        inputs = tf.keras.layers.Concatenate()([conv_outputs, inputs])
        num_filters += growth_rate # To increase the number of filters for each layer.
    return inputs, num_filters

In [ ]:

input_shape = ( 32 , 32 , 3 )
num_blocks = 3
num_layers_per_block = 4
growth_rate = 16
dropout_rate = 0.4
compress_factor = 0.5
eps = 1.1e-5

num_filters = 16

inputs = tf.keras.layers.Input( shape=input_shape )
x = tf.keras.layers.Conv2D( num_filters , kernel_size=( 3 , 3 ) , use_bias=False, kernel_initializer='he_normal' , kernel_regularizer=tf.keras.regularizers.l2( 1e-4 ) )( inputs )

for i in range( num_blocks ):
    x, num_filters = dense_block( x, num_layers_per_block , num_filters, growth_rate , dropout_rate )
    x = transition(x, num_filters , compress_factor , dropout_rate )

## GlobalAveragePooling2D layer ensures that the outputs are 2D
x = tf.keras.layers.GlobalAveragePooling2D()( x )

x = tf.keras.layers.Dense( 10 )( x ) # Num Classes for CIFAR-10
outputs = tf.keras.layers.Activation( 'softmax' )( x )


In [ ]:
## Compile the model with optimizer as 'adam' and loss function as 'categorical_crossentropy'
model = tf.keras.models.Model( inputs , outputs )
model.compile( loss='categorical_crossentropy' ,optimizer='adam' ,metrics=[ 'acc' ])
#model.summary()


In [ ]:
batch_size = 128
epochs = 200
## Train the model over 200 epochs and batch size as 128
model.fit( x_train , y_train , epochs=epochs , batch_size=batch_size , validation_data=( x_test , y_test ) )


Epoch 1/200
391/391 [==============================] - 58s 149ms/step - loss: 1.4749 - acc: 0.4520 - val_loss: 2.1544 - val_acc: 0.3936
Epoch 2/200
391/391 [==============================] - 58s 148ms/step - loss: 1.2829 - acc: 0.5352 - val_loss: 2.5570 - val_acc: 0.3822
Epoch 3/200
391/391 [==============================] - 58s 148ms/step - loss: 1.1702 - acc: 0.5834 - val_loss: 1.5393 - val_acc: 0.5521
Epoch 4/200
391/391 [==============================] - 58s 149ms/step - loss: 1.0951 - acc: 0.6121 - val_loss: 1.7217 - val_acc: 0.5277
Epoch 5/200
391/391 [==============================] - 58s 148ms/step - loss: 1.0428 - acc: 0.6302 - val_loss: 1.9355 - val_acc: 0.5391
Epoch 6/200
391/391 [==============================] - 58s 148ms/step - loss: 0.9958 - acc: 0.6476 - val_loss: 1.5506 - val_acc: 0.5956
Epoch 7/200
391/391 [==============================] - 58s 149ms/step - loss: 0.9573 - acc: 0.6630 - val_loss: 1.6789 - val_acc: 0.5858
Epoch 8/200
391/391 [===========================

In [ ]:
## Print the loss and accuracy on validation data
scores = model.evaluate(x_test, y_test, batch_size=batch_size)
print( 'Loss = {} and Accuracy = {} %'.format( scores[0] , scores[1] * 100 ) )

79/79 [==============================] - 3s 33ms/step - loss: 0.6979 - acc: 0.8629
Loss = 0.6978936791419983 and Accuracy = 86.29000186920166 %
